In [23]:
import glob
import pandas as pd
import json
import requests

# Importing and Loading the datasets

In [24]:
df = pd.concat([pd.read_csv(f) for f in glob.glob("./data/*.csv")], ignore_index=True)
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN


In [25]:
df.shape

(541593, 11)

# Handling Null Values

In [26]:
df.isnull().sum()

month                       0
town                        0
flat_type                   0
block                       0
street_name                 0
storey_range                0
floor_area_sqm              0
flat_model                  0
lease_commence_date         0
resale_price                0
remaining_lease        339399
dtype: int64

In [27]:
# droping remaining_lease missing values
df = df.dropna()
df.isnull().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
remaining_lease        0
dtype: int64

# MRT (Mass Rapid Transit System) Railway Transportation
*   Getting MRT coordinates for each city
*   so that we can calculate the distance afterwards using MRT coordinates



In [28]:
df['town'].unique()

array(['ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT BATOK', 'BUKIT MERAH',
       'BUKIT PANJANG', 'BUKIT TIMAH', 'CENTRAL AREA', 'CHOA CHU KANG',
       'CLEMENTI', 'GEYLANG', 'HOUGANG', 'JURONG EAST', 'JURONG WEST',
       'KALLANG/WHAMPOA', 'MARINE PARADE', 'PASIR RIS', 'PUNGGOL',
       'QUEENSTOWN', 'SEMBAWANG', 'SENGKANG', 'SERANGOON', 'TAMPINES',
       'TOA PAYOH', 'WOODLANDS', 'YISHUN'], dtype=object)

In [29]:
import requests

def get_mrt_stations():
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = """
    [out:json];
    area["ISO3166-1"="SG"][admin_level=2]->.search;
    (
      node(area.search)["station"="subway"];
      way(area.search)["station"="subway"];
      rel(area.search)["station"="subway"];
    );
    out center;
    """

    response = requests.post(overpass_url, data=overpass_query)

    if response.status_code == 200:
        data = response.json()
        mrt_stations = []

        for element in data['elements']:
            if 'tags' in element:
                name = element['tags'].get('name', 'Unknown Station')
                mrt_stations.append(name)

        return mrt_stations
    else:
        print(f"Failed to fetch MRT stations. Status code: {response.status_code}")
        return []

# Get a list of MRT stations in Singapore
list_of_mrt = get_mrt_stations()

# Print the list of MRT stations
print("List of MRT Stations:", list_of_mrt)


List of MRT Stations: ['Aljunied', 'Kallang', 'Yio Chu Kang', 'Tanah Merah', 'Bedok', 'Changi Airport', 'Redhill', 'Queenstown', 'Commonwealth', 'Dover', 'Clementi', 'Jurong East', 'Chinese Garden', 'Lakeside', 'Farrer Park', 'Outram Park', 'Clarke Quay', 'Boon Lay', 'Pioneer', 'Joo Koon', 'Stadium', 'Esplanade', 'Nicoll Highway', 'Mountbatten', 'Tai Seng', 'Telok Blangah', 'Labrador Park', 'Haw Par Villa', 'Kent Ridge', 'one-north', 'Gul Circle', 'Tuas Crescent', 'Tuas West Road', 'Tuas Link', 'Kembangan', 'Simei', 'Bukit Batok', 'Admiralty', 'Bukit Gombak', 'City Hall', 'Kranji', 'Lorong Chuan', 'Marsiling', 'Pasir Ris', 'Yew Tee', 'Yishun', 'Outram Park', 'Choa Chu Kang', 'Marymount', 'Eunos', 'Sungei Bedok', 'Xilin', 'Bencoolen', 'Khatib', 'Bukit Batok West', 'Tengah Plantation', 'Tengah Park', 'Tawas', 'Gek Poh', 'Pandan Reservoir', 'Peng Kang', 'Nanyang Crescent', 'Nanyang Gateway', 'Jurong Town Hall', 'Toh Guan', 'Jurong Pier', 'Jurong Hill', 'Tukang', 'Enterprise', 'Jurong West

In [30]:
import requests

mrt_lat = []
mrt_long = []

for i in range(0, len(list_of_mrt)):
    query_address = f'{list_of_mrt[i]}, Singapore'
    query_string = f'https://nominatim.openstreetmap.org/search?format=json&q={query_address}'

    resp = requests.get(query_string)

    if resp.status_code == 200:
        data_nominatim = resp.json()

        if data_nominatim:
            # Extract and append latitude and longitude to the respective lists
            mrt_lat.append(float(data_nominatim[0]['lat']))
            mrt_long.append(float(data_nominatim[0]['lon']))

            print(f"{query_address}, Lat: {data_nominatim[0]['lat']}, Long: {data_nominatim[0]['lon']}")
        else:
            mrt_lat.append('NotFound')
            mrt_long.append('NotFound')
            print(f"No Results for {query_address}")
    else:
        print(f"Failed to retrieve data for {query_address}. Status code: {resp.status_code}")

# Print or use the mrt_lat and mrt_long lists as needed
print("Latitude:", mrt_lat)
print("Longitude:", mrt_long)


Aljunied, Singapore, Lat: 1.3164515, Long: 103.8829087
Kallang, Singapore, Lat: 1.310759, Long: 103.866262
Yio Chu Kang, Singapore, Lat: 1.3817905, Long: 103.8449359
Tanah Merah, Singapore, Lat: 1.3272553, Long: 103.9465048
Bedok, Singapore, Lat: 1.3239765, Long: 103.930216
Changi Airport, Singapore, Lat: 1.3573382, Long: 103.9885647
Redhill, Singapore, Lat: 1.2896482, Long: 103.8167677
Queenstown, Singapore, Lat: 1.2946226, Long: 103.8060366
Commonwealth, Singapore, Lat: 1.3024428, Long: 103.7983085
Dover, Singapore, Lat: 1.3114155, Long: 103.7786108
Clementi, Singapore, Lat: 1.3151003, Long: 103.7652311
Jurong East, Singapore, Lat: 1.333108, Long: 103.7422939
Chinese Garden, Singapore, Lat: 1.3423978, Long: 103.732544
Lakeside, Singapore, Lat: 1.34425, Long: 103.7206931
Farrer Park, Singapore, Lat: 1.3124609, Long: 103.8541941
Outram Park, Singapore, Lat: 1.2804624, Long: 103.83991801312565
Clarke Quay, Singapore, Lat: 1.2887582, Long: 103.8466978
Boon Lay, Singapore, Lat: 1.3385504,

In [31]:
print(len(mrt_lat))
print(len(mrt_long))

240
240


In [10]:
'''mrt_lat = []
mrt_long = []

for i in range(0, len(list_of_mrt)):
    query_address = list_of_mrt[i]
    query_string = 'https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)

    data_mrt=json.loads(resp.content)

    if data_mrt['found'] != 0:
        mrt_lat.append(data_mrt["results"][0]["LATITUDE"])
        mrt_long.append(data_mrt["results"][0]["LONGITUDE"])

        print (str(query_address)+",Lat: "+data_mrt['results'][0]['LATITUDE'] +" Long: "+data_mrt['results'][0]['LONGITUDE'])

    else:
        mrt_lat.append('NotFound')
        mrt_long.append('NotFound')
        print ("No Results")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [33]:
mrt_location = pd.DataFrame({
    'MRT': list_of_mrt,
    'latitude': mrt_lat,
    'longitude': mrt_long
})

# Getting coordinates of each HDB(Housing and Developing Board) Resale flat
in order to conduct the distance from MRT stations as well as

*   So, that we can take out the distance of flats from MRT stations (Mass Rapid Transit System).
*   And, also from CBD (Central Business District).


In [34]:
df['address'] = df['block'] + " " + df['street_name']
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,address
287196,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,255000.0,70,174 ANG MO KIO AVE 4
287197,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,275000.0,65,541 ANG MO KIO AVE 10
287198,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,285000.0,64,163 ANG MO KIO AVE 4
287199,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,290000.0,63,446 ANG MO KIO AVE 10
287200,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,290000.0,64,557 ANG MO KIO AVE 10
...,...,...,...,...,...,...,...,...,...,...,...,...
489385,2023-10,YISHUN,5 ROOM,513B,YISHUN ST 51,10 TO 12,113.0,Improved,2018,715000.0,93 years 05 months,513B YISHUN ST 51
489386,2023-10,YISHUN,EXECUTIVE,406,YISHUN AVE 6,07 TO 09,148.0,Maisonette,1988,830000.0,63 years 11 months,406 YISHUN AVE 6
489387,2023-10,YISHUN,EXECUTIVE,391,YISHUN AVE 6,04 TO 06,142.0,Apartment,1988,753000.0,63 years 08 months,391 YISHUN AVE 6
489388,2023-10,YISHUN,EXECUTIVE,643,YISHUN ST 61,10 TO 12,142.0,Apartment,1987,808000.0,63 years,643 YISHUN ST 61


In [35]:
address_list = df['address'].unique()

In [36]:
import requests
import json

latitude = []
longitude = []
blk_no = []
road_name = []
postal_code = []
address = []

# Limiting to the first 300 addresses
for row in range(min(300, len(address_list))):
    query_address = address_list[row]
    query_string = f'https://nominatim.openstreetmap.org/search?format=json&q={query_address}'

    resp = requests.get(query_string)

    if resp.status_code == 200:
        data_geo_location = resp.json()

        if data_geo_location:
            # Extracting location information
            latitude.append(float(data_geo_location[0]['lat']))
            longitude.append(float(data_geo_location[0]['lon']))
            
            # Extracting address components if available
            components = data_geo_location[0].get('address', {})

            blk_no.append(components.get('house_number', ''))
            road_name.append(components.get('road', ''))
            postal_code.append(components.get('postcode', ''))
            address.append(query_address)

            print(f"{query_address}, Lat: {data_geo_location[0]['lat']}, Long: {data_geo_location[0]['lon']}")
        else:
            print(f"No Results for {query_address}")
    else:
        print(f"Failed to retrieve data for {query_address}. Status code: {resp.status_code}")

# Print or use the latitude, longitude, blk_no, road_name, postal_code, and address lists as needed


174 ANG MO KIO AVE 4, Lat: 1.38391, Long: 103.837471
541 ANG MO KIO AVE 10, Lat: 1.3739835500000002, Long: 103.85559074965985
163 ANG MO KIO AVE 4, Lat: 1.3737366, Long: 103.8376671
446 ANG MO KIO AVE 10, Lat: 1.3677793999999999, Long: 103.85533443841078
557 ANG MO KIO AVE 10, Lat: 1.37165465, Long: 103.85775588826617
603 ANG MO KIO AVE 5, Lat: 1.3801792, Long: 103.83575158255286
709 ANG MO KIO AVE 8, Lat: 1.37109375, Long: 103.84766290778387
333 ANG MO KIO AVE 1, Lat: 1.3613534500000002, Long: 103.85168816395225
109 ANG MO KIO AVE 4, Lat: 1.38391, Long: 103.837471
564 ANG MO KIO AVE 3, Lat: 1.3698918500000001, Long: 103.85943216489414
218 ANG MO KIO AVE 1, Lat: 1.3651411, Long: 103.84172788267786
556 ANG MO KIO AVE 10, Lat: 1.3720322999999999, Long: 103.85760484542894
156 ANG MO KIO AVE 4, Lat: 1.3755022499999998, Long: 103.83995684140612
471 ANG MO KIO AVE 10, Lat: 1.3635153500000001, Long: 103.8567036832668
434 ANG MO KIO AVE 10, Lat: 1.3679009500000001, Long: 103.85345464052526
560

In [67]:
import requests

latitude = []
longitude = []
blk_no = []
postal_code = []
address = []

# Limiting to the first 300 addresses
for row in range(min(300, len(address_list))):
    query_address = address_list[row]
    query_string = f'https://nominatim.openstreetmap.org/search?format=json&q={query_address}'

    resp = requests.get(query_string)

    if resp.status_code == 200:
        data_geo_location = resp.json()

        if data_geo_location:
            # Extracting location information
            latitude.append(float(data_geo_location[0]['lat']))
            longitude.append(float(data_geo_location[0]['lon']))

            # Extracting block number and postal code from display_name
            display_name_parts = data_geo_location[0]['display_name'].split(', ')
            block_code = display_name_parts[0].strip()
            postal_code_part = display_name_parts[-2].strip()

            # Extracting numeric parts from the block code and postal code part
            blk_no.append(''.join([c for c in block_code if c.isdigit()]))
            postal_code.append(''.join([c for c in postal_code_part if c.isdigit()]))

            address.append(query_address)

            print(f"{query_address}, Lat: {data_geo_location[0]['lat']}, Long: {data_geo_location[0]['lon']}, "
                  f"Block: {blk_no[-1]}, Postal Code: {postal_code[-1]}")
        else:
            print(f"No Results for {query_address}")
    else:
        print(f"Failed to retrieve data for {query_address}. Status code: {resp.status_code}")

# Print or use the latitude, longitude, blk_no, postal_code, and address lists as needed


174 ANG MO KIO AVE 4, Lat: 1.3840206, Long: 103.8372505, Block: 4, Postal Code: 787082
541 ANG MO KIO AVE 10, Lat: 1.3739835500000002, Long: 103.85559074965985, Block: 541, Postal Code: 560541
163 ANG MO KIO AVE 4, Lat: 1.373602, Long: 103.8376182, Block: , Postal Code: 569897
446 ANG MO KIO AVE 10, Lat: 1.3677793999999999, Long: 103.85533443841078, Block: 446, Postal Code: 560446
557 ANG MO KIO AVE 10, Lat: 1.37165465, Long: 103.85775588826617, Block: 557, Postal Code: 560557
603 ANG MO KIO AVE 5, Lat: 1.3801792, Long: 103.83575158255286, Block: 603, Postal Code: 560603
709 ANG MO KIO AVE 8, Lat: 1.37109375, Long: 103.84766290778387, Block: 709, Postal Code: 560709
333 ANG MO KIO AVE 1, Lat: 1.3613534500000002, Long: 103.85168816395225, Block: 333, Postal Code: 560333
109 ANG MO KIO AVE 4, Lat: 1.3840206, Long: 103.8372505, Block: 4, Postal Code: 787082
564 ANG MO KIO AVE 3, Lat: 1.3698918500000001, Long: 103.85943216489414, Block: 564, Postal Code: 560564
218 ANG MO KIO AVE 1, Lat: 1

In [19]:
'''
latitude = []
longitude = []
blk_no = []
road_name = []
postal_code = []
address = []
count = 0

for row in range(len(address_list)):
    query_address = address_list[row]
    query_string='https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)


    data_geo_location=json.loads(resp.content)
    if data_geo_location['found'] != 0:
        latitude.append(data_geo_location['results'][0]['LATITUDE'])
        longitude.append(data_geo_location['results'][0]['LONGITUDE'])
        blk_no.append(data_geo_location['results'][0]['BLK_NO'])
        road_name.append(data_geo_location['results'][0]['ROAD_NAME'])
        postal_code.append(data_geo_location['results'][0]['POSTAL'])
        address.append(query_address)
        print (str(query_address) + " ,Lat: " + data_geo_location['results'][0]['LATITUDE'] + " Long: " + data_geo_location['results'][0]['LONGITUDE'])
    else:
        print ("No Results")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [69]:
df_coordinates = pd.DataFrame({
    'latitude': latitude,
    'longitude': longitude,
    'blk_no': blk_no,
#    'road_name': road_name,
    'postal_code': postal_code,
    'address': address
})
len(df_coordinates)

186

In [70]:
list_of_lat = df_coordinates['latitude']
list_of_long = df_coordinates['longitude']
mrt_lat = mrt_location['latitude']
mrt_long = mrt_location['longitude']

In [71]:
list_of_coordinates = []
list_of_mrt_coordinates = []

for lat, long in zip(list_of_lat, list_of_long):
    list_of_coordinates.append((lat,long))
for lat, long in zip(mrt_lat, mrt_long):
    list_of_mrt_coordinates.append((lat, long))

In [72]:
from geopy.distance import geodesic

list_of_dist_mrt = []
min_dist_mrt = []

for origin in list_of_coordinates:
    valid_destinations = [dest for dest in list_of_mrt_coordinates if dest != 'NotFound']

    if valid_destinations:  # Check if there are valid destinations
        for destination in valid_destinations:
            try:
                distance = geodesic(origin, destination).meters
                list_of_dist_mrt.append(distance)
            except ValueError:
                # Handle cases where geodesic fails to calculate distance
                pass

        if list_of_dist_mrt:
            shortest = min(list_of_dist_mrt)
            min_dist_mrt.append(shortest)
        else:
            # Handle the case where no valid distances were calculated
            min_dist_mrt.append(None)

        list_of_dist_mrt.clear()
    else:
        # Handle the case where there are no valid destinations
        min_dist_mrt.append(None)

# Now min_dist_mrt contains the distances or None for cases where no valid distance was calculated


# Getting the diatnce with the help of OpenMap API

In [41]:
'''# Distance to nearest MRT
from geopy.distance import geodesic

list_of_dist_mrt = []
min_dist_mrt = []

for origin in list_of_coordinates:
    for destination in range(0, len(list_of_mrt_coordinates)):
        list_of_dist_mrt.append(geodesic(origin,list_of_mrt_coordinates[destination]).meters)
    shortest = (min(list_of_dist_mrt))
    min_dist_mrt.append(shortest)
    list_of_dist_mrt.clear()

ValueError: could not convert string to float: 'NotFound'

In [73]:
# Distance from CDB
cbd_dist = []

for origin in list_of_coordinates:
    cbd_dist.append(geodesic(origin,(1.2830, 103.8513)).meters) #CBD coordinates

In [75]:
# Put MRT and CBD distance together
df_coordinates['cbd_dist'] = cbd_dist
df_coordinates['min_dist_mrt'] = min_dist_mrt

In [76]:
df_coordinates

,latitude,longitude,blk_no,postal_code,address,cbd_dist,min_dist_mrt
0,1.384021,103.837250,4,787082,174 ANG MO KIO AVE 4,1.127924e+04,1.132490e+02
1,1.373984,103.855591,541,560541,541 ANG MO KIO AVE 10,1.007182e+04,8.014329e+02
2,1.373602,103.837618,,569897,163 ANG MO KIO AVE 4,1.013335e+04,1.905629e+02
3,1.367779,103.855334,446,560446,446 ANG MO KIO AVE 10,9.385217e+03,6.950077e+02
4,1.371655,103.857756,557,560557,557 ANG MO KIO AVE 10,9.829270e+03,5.775059e+02
...,...,...,...,...,...,...,...
181,1.385485,103.759896,626,670626,626 SENJA RD,1.522810e+04,9.093257e+02
182,1.382910,103.761370,608,677742,608 SENJA RD,1.490685e+04,5.844993e+02
183,1.378896,103.763601,,677743,182 JELEBU RD,1.441172e+04,1.220181e+02
184,1.385587,103.758700,,670627,606 SENJA RD,1.532566e+04,9.789245e+02


In [77]:
df_coordinates.to_csv('df_coordinates.csv',index=False)

In [78]:
df_coordinates = pd.read_csv('df_coordinates.csv')
df_coordinates

,latitude,longitude,blk_no,postal_code,address,cbd_dist,min_dist_mrt
0,1.384021,103.837250,4.0,787082,174 ANG MO KIO AVE 4,1.127924e+04,1.132490e+02
1,1.373984,103.855591,541.0,560541,541 ANG MO KIO AVE 10,1.007182e+04,8.014329e+02
2,1.373602,103.837618,NaN,569897,163 ANG MO KIO AVE 4,1.013335e+04,1.905629e+02
3,1.367779,103.855334,446.0,560446,446 ANG MO KIO AVE 10,9.385217e+03,6.950077e+02
4,1.371655,103.857756,557.0,560557,557 ANG MO KIO AVE 10,9.829270e+03,5.775059e+02
...,...,...,...,...,...,...,...
181,1.385485,103.759896,626.0,670626,626 SENJA RD,1.522810e+04,9.093257e+02
182,1.382910,103.761370,608.0,677742,608 SENJA RD,1.490685e+04,5.844993e+02
183,1.378896,103.763601,NaN,677743,182 JELEBU RD,1.441172e+04,1.220181e+02
184,1.385587,103.758700,NaN,670627,606 SENJA RD,1.532566e+04,9.789245e+02


In [79]:
df_new = df_coordinates.merge(df, on="address", how='outer')
df_new

,latitude,longitude,blk_no,postal_code,address,cbd_dist,min_dist_mrt,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1.384021,103.83725,4.0,787082.0,174 ANG MO KIO AVE 4,11279.239026,113.248997,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,255000.0,70
1,1.384021,103.83725,4.0,787082.0,174 ANG MO KIO AVE 4,11279.239026,113.248997,2015-12,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,275000.0,69
2,1.384021,103.83725,4.0,787082.0,174 ANG MO KIO AVE 4,11279.239026,113.248997,2016-05,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,310000.0,68
3,1.384021,103.83725,4.0,787082.0,174 ANG MO KIO AVE 4,11279.239026,113.248997,2016-06,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,253000.0,68
4,1.384021,103.83725,4.0,787082.0,174 ANG MO KIO AVE 4,11279.239026,113.248997,2016-11,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,290000.0,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202189,NaN,NaN,NaN,NaN,182B WOODLANDS ST 13,NaN,NaN,2023-10,WOODLANDS,5 ROOM,182B,WOODLANDS ST 13,10 TO 12,113.0,Improved,2019,720000.0,95 years
202190,NaN,NaN,NaN,NaN,462C YISHUN AVE 6,NaN,NaN,2023-10,YISHUN,2 ROOM,462C,YISHUN AVE 6,07 TO 09,47.0,Model A,2019,338889.0,94 years 11 months
202191,NaN,NaN,NaN,NaN,462C YISHUN AVE 6,NaN,NaN,2023-10,YISHUN,2 ROOM,462C,YISHUN AVE 6,13 TO 15,47.0,Model A,2019,320000.0,94 years 10 months
202192,NaN,NaN,NaN,NaN,462C YISHUN AVE 6,NaN,NaN,2023-10,YISHUN,3 ROOM,462C,YISHUN AVE 6,10 TO 12,68.0,Model A,2019,460000.0,94 years 11 months


In [80]:
df_new['resale_price'] = df_new['resale_price'].astype('float')
df_new['floor_area_sqm'] = df_new['floor_area_sqm'].astype('float')
df_new['lease_commence_date'] = df_new['lease_commence_date'].astype('int64')
df_new['lease_remain_years'] = 99 - (2023 - df_new['lease_commence_date'])

df_new.dropna(inplace=True)

df_new

,latitude,longitude,blk_no,postal_code,address,cbd_dist,min_dist_mrt,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,lease_remain_years
0,1.384021,103.837250,4.0,787082.0,174 ANG MO KIO AVE 4,1.127924e+04,1.132490e+02,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,255000.0,70,62
1,1.384021,103.837250,4.0,787082.0,174 ANG MO KIO AVE 4,1.127924e+04,1.132490e+02,2015-12,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,275000.0,69,62
2,1.384021,103.837250,4.0,787082.0,174 ANG MO KIO AVE 4,1.127924e+04,1.132490e+02,2016-05,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,310000.0,68,62
3,1.384021,103.837250,4.0,787082.0,174 ANG MO KIO AVE 4,1.127924e+04,1.132490e+02,2016-06,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,253000.0,68,62
4,1.384021,103.837250,4.0,787082.0,174 ANG MO KIO AVE 4,1.127924e+04,1.132490e+02,2016-11,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,290000.0,68,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4804,41.732265,-73.406393,478.0,6785.0,478 SEGAR RD,1.523223e+07,1.521356e+07,2021-11,BUKIT PANJANG,4 ROOM,478,SEGAR RD,04 TO 06,92.0,Premium Apartment,2003,425000.0,80 years 03 months,79
4805,41.732265,-73.406393,478.0,6785.0,478 SEGAR RD,1.523223e+07,1.521356e+07,2022-03,BUKIT PANJANG,4 ROOM,478,SEGAR RD,07 TO 09,92.0,Premium Apartment,2003,435000.0,79 years 11 months,79
4806,41.732265,-73.406393,478.0,6785.0,478 SEGAR RD,1.523223e+07,1.521356e+07,2022-06,BUKIT PANJANG,5 ROOM,478,SEGAR RD,01 TO 03,112.0,Premium Apartment,2003,450000.0,79 years 08 months,79
4807,41.732265,-73.406393,478.0,6785.0,478 SEGAR RD,1.523223e+07,1.521356e+07,2023-06,BUKIT PANJANG,5 ROOM,478,SEGAR RD,10 TO 12,110.0,Premium Apartment,2003,562000.0,78 years 08 months,79


# Final Combined Data

In [81]:
df_new.to_csv('combined.csv', index = False)